In [ ]:
#from langchain_groq import ChatGroq
#llm = ChatGroq(
#    model_name="qwen/qwen3-32b",  #llama-3.3-70b-versatile
#    temperature=0,
#   api_key=os.getenv("GROQ_API_KEY") 
#)

# Initialize Ollama LLaMA 3.3 Model

This snippet shows how to set up the Ollama LLaMA 3.3 language model using LangChain.

It's a simple initialization that gets the model ready for generating text or handling chat-based tasks.



In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.3")


### Setting Up MCP Client and Tools with LangGraph

This example shows how to:

- Configure an MCP client to connect to a server.
- Load available tools from the MCP server.
- Optionally preload specific domain resources.
- Bind the loaded tools to a language model for use in conversations.

Everything runs asynchronously, so you need to run it inside an async function or environment.

This setup allows your language model to use external tools and data seamlessly during interactions.



In [42]:
import asyncio
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

from langchain_mcp_adapters.client import MultiServerMCPClient

# Async setup in a function so we can use `await`

# 🧠 MCP client config
client = MultiServerMCPClient({
    "diabetes_server": {
        "url": "http://localhost:8000/mcp",
        "transport": "streamable_http"
    }
})

# 🛠️ Load tools from MCP server
mcp_tools = await client.get_tools()

# 🔍 Optional: preload domain-specific resources
resources = await client.get_resources("diabetes_server", uris=["diabetes://guidelines/risk-factors"])
print("Loaded resources:", resources)



# 🔗 Model with prompt and tool binding
model = llm.bind_tools(mcp_tools)




Loaded resources: [Blob 4525021264]


### LangGraph Chat Flow with Model and Tools

This example shows how to build a simple chat flow using LangGraph:

- The system sends user messages to a language model.
- If the model requests external tools, those tools are called.
- The results from the tools are sent back to the model.
- This process repeats until no more tool calls are needed.

The flow starts with the model generating a response and then decides whether to call tools or end the conversation.

You can run queries through this chat flow asynchronously, and get dynamic, tool-enhanced answers from the model.


In [ ]:

# LangGraph step functions
def should_continue(state):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

# Build the LangGraph
builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_node("tools", ToolNode(mcp_tools))  # Use ToolNode directly
builder.add_edge(START, "call_model")
builder.add_conditional_edges("call_model", should_continue, ["tools", END])
builder.add_edge("tools", "call_model")
graph = builder.compile()

# Run queries
print("Query 1: Predict diabetes")
result = await graph.ainvoke({  # Use ainvoke for async execution
    "messages": [{"role": "user", "content": "Predict diabetes risk for age 45, BMI 28, pedigree 0.5"}]
})
print(result["messages"][-1].content)

Query 1: Predict diabetes
The prediction is 0, which means the model predicts that the individual does not have diabetes. The probability of this prediction is 0.31, which means there is a 31% chance that the individual does not have diabetes.



### Running a Query Without Tool Invocation

This example shows a question being asked where no external tools are triggered. The language model handles the response entirely on its own without invoking any tools.



In [40]:
result = graph.invoke({
    "messages": [{"role": "user", "content": "what is an insulin"}]
})
print(result["messages"][-1].content)



Insulin is a hormone produced by the pancreas that plays a crucial role in regulating blood sugar levels. It helps to facilitate the uptake of glucose by cells throughout the body, thereby lowering blood sugar levels. Insulin is essential for maintaining normal glucose metabolism and preventing conditions such as diabetes.

In people with diabetes, the body either does not produce enough insulin (Type 1 diabetes) or is unable to effectively use the insulin it produces (Type 2 diabetes). As a result, blood sugar levels can become elevated, leading to a range of complications.

Insulin therapy is often used to manage diabetes, particularly in people with Type 1 diabetes. There are several types of insulin, including:

1. Rapid-acting insulin: Begins to work within 15-30 minutes and lasts for 2-4 hours.
2. Short-acting insulin: Begins to work within 30-60 minutes and lasts for 4-6 hours.
3. Intermediate-acting insulin: Begins to work within 1-2 hours and lasts for 12-18 hours.
4. Long-act

In [49]:
result

{'messages': [HumanMessage(content='what is a french population', additional_kwargs={}, response_metadata={}, id='58fa2bba-f94f-49fa-8cc6-797dcbf89b3e'),
  AIMessage(content="The French population refers to the people who live in France, a country located in Western Europe. As of 2021, the estimated population of France is approximately 67.2 million people. The French population is diverse and includes people of various ethnic, cultural, and linguistic backgrounds.\n\nThe French population can be broken down into several categories, including:\n\n1. Native French: People who were born in France and have French ancestry.\n2. Immigrant population: People who were born outside of France but have moved to the country, including those from European countries, Africa, Asia, and the Americas.\n3. Overseas French: People who live in French overseas departments and territories, such as Guadeloupe, Martinique, and Réunion.\n\nThe French population is known for its rich cultural heritage, includi